In [ ]:
#Transformers installation
!pip install transformers datasets
!pip install datasets transformers rouge-score nltk
!pip install transformers
!pip install pytorch_lightning
!pip install sentencepiece

In [22]:
#Change the name
from datasets import Dataset, DatasetDict
ds_from_disk = DatasetDict.load_from_disk(r'test_train_file')

In [23]:
from datasets import load_dataset, load_metric

metric = load_metric("rouge")

In [24]:
model_checkpoint = 'mrm8488/t5-base-finetuned-wikiSQL'

In [25]:
if model_checkpoint in ["mrm8488/t5-base-finetuned-wikiSQL"]:
    prefix = "translate English to SQL: %s </s>"
else:
    prefix = ""

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")

max_input_length = 3000
max_target_length = 3000


def tokenize_function(examples):
    inputs = [prefix+doc for doc in examples["question"]]
    model_inputs =  tokenizer(inputs, max_length=max_input_length, truncation=True)
    

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["sql_query"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_datasets = ds_from_disk.map(tokenize_function, batched=True, load_from_cache_file=False)

In [ ]:
tokenize_function(tokenized_datasets['train'][:2])

In [28]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['question', 'sql_query', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 16295
    })
    test: Dataset({
        features: ['question', 'sql_query', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1808
    })
})

In [29]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42)
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42)

In [30]:
small_train_dataset = small_train_dataset.remove_columns(["question", "sql_query"])
small_eval_dataset = small_eval_dataset.remove_columns(["question", "sql_query"])

In [31]:
# small_train_dataset = small_train_dataset.remove_columns(["question", "sql.human_readable"])
# small_eval_dataset = small_eval_dataset.remove_columns(["question", "sql.human_readable"])

In [32]:
small_train_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 16295
})

In [ ]:
#from transformers import AutoModelForSequenceClassification 
#model = AutoModelForSequenceClassification.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL", num_labels=5)

#from transformers import AutoModelWithLMHead
#model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")


# from transformers import T5Config, T5ForConditionalGeneration
# config = T5Config.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
# config.xla_device = False
# model = T5ForConditionalGeneration.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL", config=config)

# from transformers import T5Config, AutoModelWithLMHead
# config = T5Config.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")
# config.xla_device = False
# model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL", config=config)



from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("mrm8488/t5-base-finetuned-wikiSQL")



In [34]:
batch_size = 10
model_name = "mrm8488/t5-base-finetuned-wikiSQL"
training_args = Seq2SeqTrainingArguments(
                f"{model_name}-finetuned-xsum",
                evaluation_strategy = "epoch",
                learning_rate=2e-5,
                per_device_train_batch_size=batch_size,
                per_device_eval_batch_size=batch_size,
                weight_decay=0.01,
                save_total_limit=3,
                num_train_epochs=20,
                predict_with_generate=True,
                fp16=True,
                push_to_hub=False,
                logging_steps = 1,
  )

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [35]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [36]:
import nltk 
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
import numpy as np

In [38]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [39]:
trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    
)

Using cuda_amp half precision backend


In [ ]:
#Training will take hours
trainer.train()

In [ ]:
trainer.save_model(r'mrm8488-finetuned-1369-training')

# Running the query with the newly created model

In [2]:
from transformers import AutoModelWithLMHead, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(r"mrm8488-finetuned-1369-training")
model = AutoModelWithLMHead.from_pretrained(r"mrm8488-finetuned-1369-training")

def get_sql(query):
    
    input_text = "translate English to SQL: %s </s>" % query
    features = tokenizer([input_text], return_tensors='pt')

    output = model.generate(max_length=500,input_ids=features['input_ids'], 
               attention_mask=features['attention_mask'])
    return tokenizer.decode(output[0])


c:\Program Files\Python39\lib\site-packages\transformers\models\auto\modeling_auto.py:907: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [ ]:
query = "Enter sample query"

print(get_sql(query))